In [3]:
import argparse

In [ ]:
parser = argparse.ArgumentParser()
##### Determine dataset path here!!
parser.add_argument("path", type=str, help="path to the dataset")
parser.add_argument("--device", type=str, default="cuda: 0")
parser.add_argument("--size", type=int, default=256, help="image size")
parser.add_argument("--batch", type=int, default=16, help="batch size")
parser.add_argument("--lr", type=float, default=1e-3, help="learning rate")
parser.add_argument("--epochs", type=int, default=1e6, help="train epochs")
parser.add_argument("--num_basis", type=int, default=6, help="subspace dimension for a generator layer")
parser.add_argument("--noise_dim", type=int, default=512, help="noise dimension for the input lay of generator")
parser.add_argument("--base_channels", type=int, default=16, help="number of base channels for generator/discriminator")
parser.add_argument("--max_channels", type=int, default=512, help="maximum number of channels for generator/discriminator")
